In [1]:
import csv
from faker import Faker
import random

fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])
    
    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)
        
        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")

Сгенерировано 100000 записей и сохранено в web_server_logs.csv


In [28]:
from pyspark.sql import SparkSession   
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, \
  StringType, IntegerType, TimestampType
from pyspark.sql.functions import to_date

spark = SparkSession.builder \
  .appName('read') \
  .getOrCreate()

log_schema = StructType([
  StructField('ip', StringType(), True),
  StructField('timestamp', TimestampType(), True),
  StructField('method', StringType(), True),
  StructField('url', StringType(), True),
  StructField('response_code', IntegerType(), True),
  StructField('response_size', IntegerType(), True)
])

df_logs = spark.read.csv('web_server_logs.csv', header=True, schema=log_schema)

"""
# Общая информация о наборе данных
print('Схема данных:')
df_logs.printSchema()

# print('Первые пару записей:')
df_logs.show(5)
"""

# 1 Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP.
(
    df_logs
        .groupBy('ip')
        .agg({'ip': 'count'})
        .withColumnRenamed('count(ip)', 'request count')
        .orderBy(col('request count').desc())
).show(10)

# 2 Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.
(
    df_logs
        .groupBy('method')
        .agg({'ip': 'count'})
        .withColumnRenamed('count(ip)', 'count')
        .orderBy(col('count').desc())
).show()

# 3 Профильтруйте и посчитайте количество запросов с кодом ответа 404.
count_error_logs = (
    df_logs
        .filter(col('response_code') == 404)
        .count()
)
print(f'Количество запросов с кодом ответа 404: {count_error_logs}')

# 4 Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.
(
    df_logs
        .groupBy(to_date(col('timestamp')).alias('date'))
        .agg({'response_size': 'sum'})
        .withColumnRenamed('sum(response_size)', 'total response size')
        .orderBy(col('date'))
).show()

spark.stop()

+---------------+-------------+
|             ip|request count|
+---------------+-------------+
|  73.44.126.127|            2|
| 131.215.66.135|            2|
|  213.72.49.169|            2|
|205.171.189.246|            2|
| 171.21.137.175|            2|
| 163.71.103.243|            1|
| 89.163.204.199|            1|
|  55.128.219.27|            1|
| 204.29.115.200|            1|
| 146.157.80.115|            1|
+---------------+-------------+
only showing top 10 rows

+------+-----+
|method|count|
+------+-----+
|  POST|25186|
|   GET|25031|
|   PUT|24972|
|DELETE|24811|
+------+-----+

Количество запросов с кодом ответа 404: 24899
+----------+-------------------+
|      date|total response size|
+----------+-------------------+
|2024-01-01|            2311166|
|2024-01-02|            2562197|
|2024-01-03|            2143875|
|2024-01-04|            2364720|
|2024-01-05|            2248246|
|2024-01-06|            2284276|
|2024-01-07|            2198608|
|2024-01-08|            23718